# Analytic "dashboard" notebook to do some basic analysis on the song play data

In [ ]:
import os
os.chdir('..')
import psycopg2
from psycopg2 import sql
import pandas as pd
from sql_queries import *
import matplotlib.pyplot as plt
import seaborn as sns
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()
print(conn)

### Explore the solution dataset (the songplays table) where the artist and song columns are complete

In [ ]:
query =("SELECT * FROM songplays \
        WHERE song_id IS NOT NULL \
        OR artist_id IS NOT NULL \
        ORDER BY start_time;")
cur.execute(query)
colnames = [desc[0] for desc in cur.description]
results = pd.DataFrame(cur.fetchall(), columns = colnames)
results

There are **333 rows** in the songplays table with complete information (ie. 333 events where a song from the events data matches a song from the songs data). This should be sufficient for some basic analysis.

### What are the top artists from the sparkify data? What are the top artists for just the paid users?
### *Run this section twice to get the proper figure to display*    
Note: ignore the font warnings when plotting, they seem irrelevant!

In [ ]:
# Top artists for all users
query = ("""
    SELECT a.name AS artist_name, COUNT(*) AS num_plays 
    FROM songplays s 
    JOIN artists a
    ON s.artist_id = a.artist_id
    GROUP BY a.name 
    ORDER BY num_plays DESC 
    LIMIT 10;
""")
cur.execute(query)
colnames = [desc[0] for desc in cur.description]
results1 = pd.DataFrame(cur.fetchall(), columns = colnames)
#print(results1)

# Top artists just for paid users
query = ("""
    SELECT a.name AS artist_name, COUNT(*) AS num_plays 
    FROM songplays s
    JOIN artists a
    ON s.artist_id = a.artist_id
    AND s.level = 'paid'
    GROUP BY a.name 
    ORDER BY num_plays DESC 
    LIMIT 10;
""")
cur.execute(query)
colnames = [desc[0] for desc in cur.description]
results2 = pd.DataFrame(cur.fetchall(), columns = colnames)
#print(results2)

# Plot results as a Seaborn barplot (2 subplots)
fig, axes = plt.subplots(2, 1)
fig.subplots_adjust(hspace=1)
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(15,18)})
sns.set(font_scale = 2)
plt.rcParams["xtick.labelsize"] = 16
chart1 = sns.barplot(x="artist_name", y="num_plays", data=results1, ax=axes[0]);
chart1.set_xticklabels(chart1.get_xticklabels(), rotation=40, horizontalalignment='right');
chart1.set_title("Top 10 artists from Sparkify dataset: All users", weight = "bold");
chart1.set(xlabel=None)
chart2 = sns.barplot(x="artist_name", y="num_plays", data=results2, ax=axes[1]);
chart2.set_title("Top 10 artists from Sparkify dataset: PAID users only", weight = "bold");
chart2.set_xticklabels(chart2.get_xticklabels(), rotation=40, horizontalalignment='right');
chart2.set(xlabel=None)

### Findings:
- No major differences in the music interests between all users and paid users (same artists, just in slightly different orders)  
- Dwight Yaokam is the most popular artist by far. This suggests many users enjoy country music.

### Suggestions for Udacity to improve project:
- Include some other fields such as song genre, and beats-per-minute, so we can do more in depth analysis of what users preferences are.

In [ ]:
cur.close()
conn.close()